In [1]:
using Pkg
Pkg.activate("/home/gribeill/GitHub/HiQuER/")
using HiQuER
using LRUCache
using BenchmarkTools
using OpenQASM
using JSON
using QuantumClifford
using Match

  Activating project at `~/GitHub/HiQuER`
┌ Info: Precompiling HiQuER [4b5eb2ad-d69c-411d-be4a-18f852c96210]
└ @ Base loading.jl:1664


In [4]:
macro bash_str(s) open(`bash`,"w",stdout) do io; print(io, s); end; end

@bash_str (macro with 1 method)

In [49]:
bash"""
staq -O2 test/plasma128.qasm | tee test/plasma4_opt.qasm | staq_resource_estimator
"""


Resources used:
  rz(-0.00134692): 2
  rz(-0.00099068): 2
  rz(-0.00115687): 2
  rz(-0.00130201): 2
  rz(-0.000344911): 2
  rz(-0.000689821): 2
  rz(-0.00124359): 4
  rz(-0.000487777): 2
  rz(-0.000597403): 2
  rz(-0.00133583): 4
  rz(-0.00113086): 2
  rz(-0.000844855): 2
  rz(-0.00119481): 4
  rz(-0.00102026): 2
  rz(-0.00132465): 2
  rz(-0.00107698): 2
  rz(-0.00114394): 4
  rz(-0.00150343): 2
  rz(-0.00158058): 2
  rz(-0.00154249): 2
  rz(-0.00161777): 2
  rz(-0.00165413): 2
  rz(-0.00175871): 2
  rz(-0.0018574): 2
  rz(-0.0014221): 2
  rz(-0.00179221): 2
  rz(-0.00037483): 2
  rz(-0.0018251): 2
  rz(-0.000430457): 2
  rz(-0.000462385): 2
  rz(-0.00118229): 2
  rz(-0.000453493): 2
  rz(-0.000983146): 2
  rz(-0.000483903): 2
  rz(-0.000369358): 2
  rz(-0.000416025): 2
  rz(-0.000308497): 2
  rz(-0.000422427): 2
  rz(-0.00135791): 4
  rz(-0.000731666): 2
  rz(-0.00192038): 2
  rz(-0.00121944): 4
  rz(-0.000488093): 2
  rz(-0.000545351): 2
  rz(-0.00131338): 4
  rz(-0.00116965): 4
  rz

In [50]:
bash"""
cat test/plasma4_opt.qasm | python3 ../util/rotation_synth/rotation_synth.py | \
tee test/plasma4_full_CT.qasm | staq_resource_estimator
"""

Resources used:
  depth: 7464
  x: 55
  y: 83
  s: 24114
  t: 43496
  cx: 2868
  h: 45498
  z: 159
  qubits: 128


In [48]:
bash"""
cat test/plasma4_full_CT.qasm | staq_lattice_surgery > test/plasma4_full_LS.json
"""

In [28]:
bash"""
cat test/plasma4_opt.qasm | staq_lattice_surgery > test/plasma4_nonCT.json
"""

LoadError: LoadError: failed process: Process(`[4mbash[24m`, ProcessSignaled(2)) [0]

in expression starting at In[28]:1

In [9]:
json = JSON.parsefile("./test/plasma4_nonCT.json")

Dict{String, Any} with 3 entries:
  "2. Circuit after the Li… => Dict{String, Any}("layers"=>Any[Dict{String, Any…
  "1. Circuit as Pauli rot… => Dict{String, Any}("layers"=>Any[Dict{String, Any…
  "3. Circuit after T dept… => nothing

In [10]:
lyrs = json["2. Circuit after the Litinski Transform"]["layers"];

lyrsT = filter(x -> !(x["pi*"] in ("1/4", "-1/4")), lyrs) 

10-element Vector{Any}:
 Dict{String, Any}("q1" => "X", "q0" => "X", "pi*" => "-0.006775", "q2" => "X", "q3" => "X")
 Dict{String, Any}("q1" => "Z", "q0" => "Z", "pi*" => "-0.003787", "q2" => "Z")
 Dict{String, Any}("q1" => "Z", "q0" => "Z", "pi*" => "0.002725", "q2" => "Z", "q3" => "Z")
 Dict{String, Any}("q1" => "X", "q0" => "X", "pi*" => "0.010754", "q2" => "X", "q3" => "X")
 Dict{String, Any}("q1" => "Z", "q0" => "Z", "pi*" => "0.006012", "q2" => "Z")
 Dict{String, Any}("q1" => "X", "q0" => "X", "pi*" => "0.010754", "q2" => "X", "q3" => "X")
 Dict{String, Any}("q1" => "Z", "q0" => "Z", "pi*" => "-0.006012", "q2" => "Z")
 Dict{String, Any}("q1" => "Z", "q0" => "Z", "pi*" => "0.002725", "q2" => "Z", "q3" => "Z")
 Dict{String, Any}("q1" => "X", "q0" => "X", "pi*" => "-0.006775", "q2" => "X", "q3" => "X")
 Dict{String, Any}("q1" => "Z", "q0" => "Z", "pi*" => "0.003787", "q2" => "Z")

In [11]:
function topaulistrings(lyrs)
    ps = []
    ang = Vector{Float64}()
    for ly in lyrs
        ops_x = zeros(Bool, 3)
        ops_z = zeros(Bool, 3)
        for j=1:3
            if !("q$(j-1)" in keys(ly))
                ops_x[j] = false
                ops_z[j] = false
                continue
            end
            ops_x[j] = @match ly["q$(j-1)"] begin
                "X" => true
                "Y" => true
                "Z" => false
            end
            ops_z[j] = @match ly["q$(j-1)"] begin
                "X" => false
                "Y" => true
                "Z" => true
            end
        end
        pop = PauliOperator(ops_x, ops_z)
        println(pop)
        push!(ps, pop)
        push!(ang, parse(Float64, ly["pi*"]) * pi)
    end
    return Dict(ps => ang)
end

topaulistrings (generic function with 1 method)

In [12]:
ps = topaulistrings(lyrsT)

+ XXX
+ ZZZ
+ ZZZ
+ XXX
+ ZZZ
+ XXX
+ ZZZ
+ ZZZ
+ XXX
+ ZZZ


Dict{Vector{Any}, Vector{Float64}} with 1 entry:
  [+ XXX, + ZZZ, + ZZZ, + … => [-0.0212843, -0.0118972, 0.00856084, 0.0337847, …

In [15]:
circ = to_circuit(ps)

1: CNOT(1, 3)*H(2)
2: S(1)*S(2)
3: S(1)*S(2)
4: H(1)*H(2)
5: CNOT(2, 3)*Rz(0.021284290228070848)(1)
6: Rz(-0.033784687396704635)(1)*Rz(0.011897211379144547)(3)
7: Rz(-0.033784687396704635)(1)
8: Rz(0.021284290228070848)(1)
9: Rz(-0.008560839981032187)(3)
10: Rz(-0.018887255033381838)(3)
11: Rz(0.018887255033381838)(3)
12: Rz(-0.008560839981032187)(3)
13: Rz(-0.011897211379144547)(3)
14: CNOT(2, 3)
15: H(1)*H(2)
16: S†(1)*S†(2)
17: S†(1)*S†(2)
18: CNOT(1, 3)*H(2)

In [16]:
open("test/small.qasm", "w") do io
    write(io, string(to_qasm(circ)))
end

542

In [20]:
bash"""
cat test/small.qasm | python3 ../util/rotation_synth/rotation_synth.py | \
tee test/small_CT.qasm | staq_resource_estimator
"""

Resources used:
  depth: 1067
  s: 366
  t: 704
  h: 718
  cx: 4
  z: 5
  qubits: 3


In [22]:
@time bash"""
cat test/small_CT.qasm | staq_lattice_surgery > test/test_ct_opt.json
"""

  0.000001 seconds
